In [1]:
import pandas as pd
import os
import numpy as np
import librosa
from sklearn.preprocessing import OneHotEncoder

In [2]:
paths=[]
labels=[]
for filename in os.listdir('./AudioWAV'):
    
    paths.append('./AudioWAV/' + filename)
    file = filename.split('.')[0]
   
    label = file.split('_')[2]
    if label == 'ANG':
        labels.append('angry.wav')
    elif label == 'DIS':
        labels.append('disgust.wav')
    elif label == 'FEA':
        labels.append('fear.wav')
    elif label == 'HAP':
        labels.append('happy.wav')
    elif label == 'NEU':
        labels.append('neutral.wav')
    elif label == 'SAD':
        labels.append('sad.wav')
        

df_cremad = pd.DataFrame({'speech':paths,'label':labels})
df_cremad.sample(5)

,speech,label
286,./AudioWAV/1004_IWL_DIS_XX.wav,disgust.wav
27,./AudioWAV/1001_IOM_SAD_XX.wav,sad.wav
424,./AudioWAV/1006_IEO_HAP_HI.wav,happy.wav
929,./AudioWAV/1012_IWL_ANG_XX.wav,angry.wav
5354,./AudioWAV/1066_IWL_HAP_XX.wav,happy.wav


In [3]:
def MFCC(filename):
    y, sr = librosa.load(filename,duration=3,offset=0.5)
    return np.mean(librosa.feature.mfcc(y=y,sr=sr,n_mfcc=40).T,axis=0)

mfcc_cremad = df_cremad['speech'].apply(lambda x:MFCC(x))

In [4]:
X =[x for x in mfcc_cremad]
X =np.array(X)
X.shape
X =np.expand_dims(X,-1)
X.shape

(7442, 40, 1)

In [5]:
ohe=OneHotEncoder()
y = ohe.fit_transform(df_cremad[['label']] )
y = y.toarray()

In [6]:
X.shape, y.shape

((7442, 40, 1), (7442, 6))

In [7]:
df_cremad['label'].unique()

array(['angry.wav', 'disgust.wav', 'fear.wav', 'happy.wav', 'neutral.wav',
       'sad.wav'], dtype=object)

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np

# Definindo os modelos

class SimpleDNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SimpleDNN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

class LSTMAttention(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMAttention, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.attention = nn.Linear(hidden_size, 1)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        out, _ = self.lstm(x, (h0, c0))
        attn_weights = torch.softmax(self.attention(out), dim=1)
        out = torch.sum(attn_weights * out, dim=1)
        out = self.fc(out)
        return out

class CNNModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(64, 32, kernel_size=3, padding=1)
        self.fc_input_size = 32 * 1 * 1
        self.fc1 = nn.Linear(self.fc_input_size, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.dropout(x)
        x = x.view(-1, self.fc_input_size)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


class CNNAttention(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNNAttention, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(64, 32, kernel_size=3, padding=1)
        self.attention = nn.Linear(32, 1)
        self.fc = nn.Linear(32, num_classes)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        attn_weights = self.softmax(self.attention(x.permute(0, 2, 1))).squeeze(-1)
        attn_weights = attn_weights.unsqueeze(-1)
        x = torch.sum(attn_weights * x, dim=2)
        x = self.fc(x)
        return x
    
# Construindo e treinando os modelos

input_size = X.shape[1:]
num_classes = y.shape[1]
hidden_size = 64
num_layers = 2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Convertendo os dados para tensores PyTorch
X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y, dtype=torch.float32).to(device)

# Definindo o tamanho do lote
batch_size = 32

# Criando conjuntos de dados PyTorch
dataset = torch.utils.data.TensorDataset(X_tensor, y_tensor)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Definindo tensor para o LSTM
X_tensorLSTM = X_tensor.permute(0, 2, 1)

# Criando conjuntos de dados PyTorch para LSTM
datasetLSTM = torch.utils.data.TensorDataset(X_tensorLSTM, y_tensor)
train_sizeLSTM = int(0.8 * len(datasetLSTM))
test_sizeLSTM = len(datasetLSTM) - train_sizeLSTM
train_datasetLSTM, test_datasetLSTM = torch.utils.data.random_split(datasetLSTM, [train_sizeLSTM, test_sizeLSTM])

# DataLoader para o LSTM
train_loaderLSTM = DataLoader(train_datasetLSTM, batch_size=batch_size, shuffle=True)
test_loaderLSTM = DataLoader(test_datasetLSTM, batch_size=batch_size, shuffle=False)

# Convertendo os dados para tensores PyTorch do CNN
X_tensorCNN = torch.tensor(X, dtype=torch.float32).to(device)

# Criando conjuntos de dados PyTorch do CNN
datasetCNN = torch.utils.data.TensorDataset(X_tensorCNN, y_tensor)
train_sizeCNN = int(0.8 * len(datasetCNN))
test_sizeCNN = len(datasetCNN) - train_sizeCNN
train_datasetCNN, test_datasetCNN = torch.utils.data.random_split(datasetCNN, [train_sizeCNN, test_sizeCNN])

# DataLoader para o CNN
train_loaderCNN = DataLoader(train_datasetCNN, batch_size=batch_size, shuffle=True)
test_loaderCNN = DataLoader(test_datasetCNN, batch_size=batch_size, shuffle=False)

# Função para treinamento
def train(model, train_loader, criterion, optimizer, num_epochs=500):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            #print(outputs.shape)
            loss = criterion(outputs, torch.max(labels, 1)[1])
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == torch.max(labels, 1)[1]).sum().item()

        epoch_loss = running_loss / len(train_loader)
        accuracy = correct / total
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}')

In [9]:
# Standard Deep Neural Network
sdnn_model = SimpleDNN(input_size[0], num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(sdnn_model.parameters(), lr=0.00001)
train(sdnn_model, train_loader, criterion, optimizer)

Epoch [1/500], Loss: 11.7832, Accuracy: 0.1745
Epoch [2/500], Loss: 10.1525, Accuracy: 0.1629
Epoch [3/500], Loss: 8.9746, Accuracy: 0.1643
Epoch [4/500], Loss: 8.0096, Accuracy: 0.1698
Epoch [5/500], Loss: 7.3057, Accuracy: 0.1685
Epoch [6/500], Loss: 6.9775, Accuracy: 0.1589
Epoch [7/500], Loss: 6.3433, Accuracy: 0.1735
Epoch [8/500], Loss: 5.8596, Accuracy: 0.1693
Epoch [9/500], Loss: 5.4722, Accuracy: 0.1653
Epoch [10/500], Loss: 5.1189, Accuracy: 0.1571
Epoch [11/500], Loss: 4.7257, Accuracy: 0.1658
Epoch [12/500], Loss: 4.4321, Accuracy: 0.1592
Epoch [13/500], Loss: 3.9661, Accuracy: 0.1676
Epoch [14/500], Loss: 3.7935, Accuracy: 0.1724
Epoch [15/500], Loss: 3.5578, Accuracy: 0.1668
Epoch [16/500], Loss: 3.3324, Accuracy: 0.1710
Epoch [17/500], Loss: 3.1212, Accuracy: 0.1666
Epoch [18/500], Loss: 2.9182, Accuracy: 0.1725
Epoch [19/500], Loss: 2.8233, Accuracy: 0.1710
Epoch [20/500], Loss: 2.7642, Accuracy: 0.1609
Epoch [21/500], Loss: 2.5490, Accuracy: 0.1740
Epoch [22/500], Loss

In [10]:
# LSTM
lstm_model = LSTMModel(input_size[0], hidden_size, num_layers, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm_model.parameters(), lr=0.00001)
train(lstm_model, train_loaderLSTM, criterion, optimizer)

Epoch [1/500], Loss: 1.7929, Accuracy: 0.1628
Epoch [2/500], Loss: 1.7925, Accuracy: 0.1655
Epoch [3/500], Loss: 1.7916, Accuracy: 0.1645
Epoch [4/500], Loss: 1.7914, Accuracy: 0.1671
Epoch [5/500], Loss: 1.7908, Accuracy: 0.1698
Epoch [6/500], Loss: 1.7899, Accuracy: 0.1771
Epoch [7/500], Loss: 1.7893, Accuracy: 0.1831
Epoch [8/500], Loss: 1.7879, Accuracy: 0.2034
Epoch [9/500], Loss: 1.7865, Accuracy: 0.2374
Epoch [10/500], Loss: 1.7844, Accuracy: 0.2705
Epoch [11/500], Loss: 1.7831, Accuracy: 0.2817
Epoch [12/500], Loss: 1.7807, Accuracy: 0.2854
Epoch [13/500], Loss: 1.7785, Accuracy: 0.2909
Epoch [14/500], Loss: 1.7761, Accuracy: 0.2935
Epoch [15/500], Loss: 1.7738, Accuracy: 0.2956
Epoch [16/500], Loss: 1.7715, Accuracy: 0.2946
Epoch [17/500], Loss: 1.7682, Accuracy: 0.2973
Epoch [18/500], Loss: 1.7646, Accuracy: 0.2980
Epoch [19/500], Loss: 1.7617, Accuracy: 0.2977
Epoch [20/500], Loss: 1.7586, Accuracy: 0.2985
Epoch [21/500], Loss: 1.7539, Accuracy: 0.3034
Epoch [22/500], Loss: 

In [11]:
# LSTM with Attention
lstm_atn_model = LSTMAttention(input_size[0], hidden_size, num_layers, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm_atn_model.parameters(), lr=0.00001)
train(lstm_atn_model, train_loaderLSTM, criterion, optimizer)

Epoch [1/500], Loss: 1.7957, Accuracy: 0.1487
Epoch [2/500], Loss: 1.7953, Accuracy: 0.1475
Epoch [3/500], Loss: 1.7941, Accuracy: 0.1505
Epoch [4/500], Loss: 1.7938, Accuracy: 0.1676
Epoch [5/500], Loss: 1.7925, Accuracy: 0.1687
Epoch [6/500], Loss: 1.7916, Accuracy: 0.1690
Epoch [7/500], Loss: 1.7909, Accuracy: 0.1692
Epoch [8/500], Loss: 1.7894, Accuracy: 0.1692
Epoch [9/500], Loss: 1.7886, Accuracy: 0.1692
Epoch [10/500], Loss: 1.7874, Accuracy: 0.1692
Epoch [11/500], Loss: 1.7862, Accuracy: 0.1692
Epoch [12/500], Loss: 1.7853, Accuracy: 0.1697
Epoch [13/500], Loss: 1.7839, Accuracy: 0.1784
Epoch [14/500], Loss: 1.7823, Accuracy: 0.2039
Epoch [15/500], Loss: 1.7811, Accuracy: 0.2303
Epoch [16/500], Loss: 1.7791, Accuracy: 0.2508
Epoch [17/500], Loss: 1.7771, Accuracy: 0.2698
Epoch [18/500], Loss: 1.7745, Accuracy: 0.2837
Epoch [19/500], Loss: 1.7715, Accuracy: 0.2960
Epoch [20/500], Loss: 1.7690, Accuracy: 0.3015
Epoch [21/500], Loss: 1.7654, Accuracy: 0.3074
Epoch [22/500], Loss: 

In [12]:
# Convolutional Neural Network
cnn_model = CNNModel(input_size[0],num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=0.00001)
train(cnn_model, train_loaderCNN, criterion, optimizer)

Epoch [1/500], Loss: 2.1224, Accuracy: 0.1608
Epoch [2/500], Loss: 1.9625, Accuracy: 0.1727
Epoch [3/500], Loss: 1.9240, Accuracy: 0.1658
Epoch [4/500], Loss: 1.8756, Accuracy: 0.1668
Epoch [5/500], Loss: 1.8542, Accuracy: 0.1666
Epoch [6/500], Loss: 1.8194, Accuracy: 0.1824
Epoch [7/500], Loss: 1.8137, Accuracy: 0.1828
Epoch [8/500], Loss: 1.8091, Accuracy: 0.1777
Epoch [9/500], Loss: 1.8013, Accuracy: 0.1816
Epoch [10/500], Loss: 1.7891, Accuracy: 0.1923
Epoch [11/500], Loss: 1.7803, Accuracy: 0.1964
Epoch [12/500], Loss: 1.7760, Accuracy: 0.2002
Epoch [13/500], Loss: 1.7729, Accuracy: 0.2065
Epoch [14/500], Loss: 1.7656, Accuracy: 0.2222
Epoch [15/500], Loss: 1.7613, Accuracy: 0.2148
Epoch [16/500], Loss: 1.7595, Accuracy: 0.2147
Epoch [17/500], Loss: 1.7570, Accuracy: 0.2343
Epoch [18/500], Loss: 1.7503, Accuracy: 0.2419
Epoch [19/500], Loss: 1.7456, Accuracy: 0.2441
Epoch [20/500], Loss: 1.7459, Accuracy: 0.2466
Epoch [21/500], Loss: 1.7418, Accuracy: 0.2478
Epoch [22/500], Loss: 

In [13]:
# Convolutional Neural Network with Attention
cnn_atn_model = CNNAttention(input_size[0],num_classes).to(device)
optimizer = optim.Adam(cnn_atn_model.parameters(), lr=0.00001)
train(cnn_atn_model, train_loaderCNN, criterion, optimizer)

Epoch [1/500], Loss: 3.8311, Accuracy: 0.1698
Epoch [2/500], Loss: 2.5819, Accuracy: 0.1698
Epoch [3/500], Loss: 2.0658, Accuracy: 0.1698
Epoch [4/500], Loss: 1.8669, Accuracy: 0.1720
Epoch [5/500], Loss: 1.8025, Accuracy: 0.2053
Epoch [6/500], Loss: 1.7774, Accuracy: 0.2699
Epoch [7/500], Loss: 1.7615, Accuracy: 0.2735
Epoch [8/500], Loss: 1.7472, Accuracy: 0.2745
Epoch [9/500], Loss: 1.7268, Accuracy: 0.2834
Epoch [10/500], Loss: 1.7102, Accuracy: 0.2926
Epoch [11/500], Loss: 1.6971, Accuracy: 0.2978
Epoch [12/500], Loss: 1.6880, Accuracy: 0.3007
Epoch [13/500], Loss: 1.6794, Accuracy: 0.3059
Epoch [14/500], Loss: 1.6700, Accuracy: 0.3099
Epoch [15/500], Loss: 1.6621, Accuracy: 0.3066
Epoch [16/500], Loss: 1.6550, Accuracy: 0.3094
Epoch [17/500], Loss: 1.6469, Accuracy: 0.3151
Epoch [18/500], Loss: 1.6397, Accuracy: 0.3131
Epoch [19/500], Loss: 1.6362, Accuracy: 0.3160
Epoch [20/500], Loss: 1.6326, Accuracy: 0.3150
Epoch [21/500], Loss: 1.6298, Accuracy: 0.3200
Epoch [22/500], Loss: 

In [14]:
# Função para teste
def test(model, test_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, torch.max(labels, 1)[1])
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == torch.max(labels, 1)[1]).sum().item()

    test_loss = running_loss / len(test_loader)
    test_accuracy = correct / total
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

In [15]:
print("Testing SimpleDNN:")
test(sdnn_model, test_loader, criterion)

Testing SimpleDNN:
Test Loss: 1.5304, Test Accuracy: 0.3526


In [16]:
print("\nTesting LSTMModel:")
test(lstm_model, test_loaderLSTM, criterion)



Testing LSTMModel:
Test Loss: 1.4163, Test Accuracy: 0.4184


In [17]:
print("\nTesting LSTMAttention:")
test(lstm_atn_model, test_loaderLSTM, criterion)


Testing LSTMAttention:
Test Loss: 1.4278, Test Accuracy: 0.4150


In [18]:
print("\nTesting CNNModel:")
test(cnn_model, test_loaderCNN, criterion)



Testing CNNModel:
Test Loss: 1.4725, Test Accuracy: 0.4150


In [19]:
print("\nTesting CNNAttention:")
test(cnn_atn_model, test_loaderCNN, criterion)


Testing CNNAttention:
Test Loss: 1.4288, Test Accuracy: 0.4150
